## Part 1
<u>*Data scraping*</u>
Initially import the modules you will need:
- pandas (dataframe)
- numpy 
- beautiful soup (to parse through the text of the url and scrape it)
- requests(to access the url)

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests


- Obtain the wikipedia url that has the neighborhoods and postcodes.
- Create the beautiful soup object. 
    - find the table in the soup object and prettify it so we can inspect it

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(wiki_url).text
soup = BeautifulSoup(data, 'html5lib')

In [3]:
table = soup.find('table')
#print(table.prettify())

- Create a list that will house the data we scrape
- go through each cell in the table on the wiki article and scrape from it the Post Code, Neighborhood and Borough.
    - When scraping the borough and neighborhood we need to split on the brackets as the name not in brackets is the Borough and the name(s) in brackets are the Neighborhood(s). 

In [4]:
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Part 2
<u>*Coordinates*</u>
<br>
Below code is edited to loop through every post code in the dataframe and receive their latitude and longitude. I kept returnign none however, so went ahead with using the csv instead. 

In [6]:
import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None
#Latitude = []
#Longitude = []

# loop until you get the coordinates
#for pc in df['PostalCode']:
    #postal_code = pc
    #print(postal_code)
   # while(lat_lng_coords is None):
        #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
       # lat_lng_coords = g.latlng
    #print(postal_code)
    #print(lat_lng_coords[0])
    #print(lat_lng_coorsd[1])
    #Latitude.append(lat_lng_coords[0])
   # Longitude.append(lat_lng_coords[1])

In [10]:
coords = pd.read_csv('C:/Users/swamp/Documents/Geospatial_Coordinates.csv')
coords

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
df = pd.merge(df, coords, on='PostalCode')
df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
